In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle as pkl
import glob
%cd ../

/mnt/c/Users/Daniel Yang/Desktop/Final_Project


In [10]:
fileId_path = "experiments/fileIds.pkl"
queryInfo_path = "experiments/queryInfo.pkl"
timeData_path = "experiments/timeData.pkl"
audioFiles_path = "experiments/audioFiles.pkl"
with open(fileId_path, 'rb') as f:
    fileIds=pkl.load(f)
with open(queryInfo_path, 'rb') as f:
    queryInfo=pkl.load(f)
with open(timeData_path, 'rb') as f:
    timeData=pkl.load(f)
with open(audioFiles_path, 'rb') as f:
    audioFiles=pkl.load(f)

In [11]:
def readHypFiles(hypdir):
    data = []
    for hypfile in sorted(glob.glob("experiments/{}/*.hyp".format(hypdir))):
        print(hypfile)
        with open(hypfile, "rb") as f:
            data.append(pkl.load(f))
    return data

In [26]:
def evaluateSegmentLevel(segment_predict, solo_name, ref_name, sr=22050):
    pieceName = solo_name.split('_')[0]
    gt_segments = queryInfo[pieceName]
    gt_samples = []
    diffs = []
    for beats in gt_segments:
        beat1 = beats.split(',')[0][1:]
        beat2 = beats.split(',')[1][:-1]
        sample1 = timeData[ref_name][beat1]*512
        sample2 = timeData[ref_name][beat2]*512
        gt_samples.append((sample1, sample2))
    for idx,segment in enumerate(segment_predict):
        seg0 = segment[0]
        seg1 = segment[1]
        diff0 = np.abs(gt_samples[idx][0]-seg0)/sr
        diff1 = np.abs(gt_samples[idx][1]-seg1)/sr
        diffs.append(diff0)
        diffs.append(diff1)
    return diffs

In [36]:
def evaluateBeatLevel(tf, segment_predict, solo_name, ref_name, sr=22050):
    # Only evaluate within segments
    diffs = []
    pieceName = solo_name.split('_')[0]
    gt_segments = queryInfo[pieceName]
    annotated_beats = sorted(list(timeData[ref_name].keys()))
    for beats in gt_segments:
        beat1 = beats.split(',')[0][1:]
        beat2 = beats.split(',')[1][:-1]
        idx1 = annotated_beats.index(beat1)
        idx2 = annotated_beats.index(beat2)
        base_time = timeData[ref_name][beat1]
        new_times = {}
        for i in range(idx1,idx2):
            key = annotated_beats[i]
            sample = (timeData[ref_name][key]-base_time)*sr
            new_time = tf[int(sample)]
            new_times[key]=new_time
        gt_beats = sorted(list(timeData[solo_name].keys()))
        gt_base_time=timeData[solo_name][beat1]
        for key in new_times:
            if key in gt_beats:
                gt_time = (timeData[solo_name][key]-gt_base_time)*sr
                pred_time = new_times[key]
                diffs.append(np.abs(gt_time-pred_time))
    return diffs

In [43]:
def evaluate(hypdir, sr=22050):
    data = readHypFiles(hypdir)
    times = []
    for solo_name, ref_name, tf, segment_predict, time_taken in data:
        segment_predict = np.array(segment_predict)*512
        segment_diffs = evaluateSegmentLevel(segment_predict, solo_name, ref_name)
        beat_diffs = evaluateBeatLevel(tf, segment_predict, solo_name, ref_name)
        segment_diffs =np.array(segment_diffs)/sr
        beat_diffs =np.array(beat_diffs)/sr
        times.append(time_taken)
        

In [44]:
evaluate("debug")

experiments/debug/p1s_perf25-p1f_perf23.hyp
[0.00055334 0.00153777 0.00322933 0.00389941 0.00387694 0.00565711
 0.00470725 0.00725587 0.00680532 0.00780399]
[4.53514739e-05 1.56130007e-01            nan            nan
 3.21587302e-01 3.56764928e-01 4.97490552e-01 6.57823129e-01
 8.52139078e-01 9.41783824e-01 1.05514739e+00 1.23814059e+00
 1.35552532e+00 1.47342404e+00 1.62149660e+00 1.77009826e+00
 4.53514739e-05 1.74603175e-01            nan            nan
 3.41980348e-01 5.86485261e-01 7.43779290e-01 9.35646259e-01
 1.06817838e+00 1.26816327e+00 1.52380952e+00 1.66433862e+00
 1.81203326e+00 1.95854875e+00 2.11662887e+00 2.24290249e+00
 2.39327286e+00 2.52837491e+00 2.28527589e+00 1.94730159e+00
 2.03286470e+00 2.10956916e+00 2.14068027e+00 1.85549509e+00
            nan            nan 2.11179138e+00 2.04869237e+00
 2.13321240e+00 2.02400605e+00 2.04612245e+00 2.10822373e+00
 2.00030234e+00 1.97720333e+00 1.54423280e+00 1.91274376e+00
 1.80922147e+00 1.83334845e+00 1.76811791e+00 2.00